In [1]:
import torch
from PIL import Image
from torchvision.transforms import v2
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from skimage.filters import butterworth
from skimage.filters import gaussian
from skimage import exposure

c:\Users\Hamad\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\Hamad\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedba

In [2]:
googleNet = torch.load('Models weights\googlenet_mura_phase_3_20epoch.pth')
googleNet = googleNet.to('cpu')


In [3]:
swin = torch.load('Models weights\SWIN_mura_phase_3_v2_7epoch.pth')
swin = swin.to('cpu')


In [39]:
image = Image.open(r"MURA-v1.1\valid\XR_WRIST\patient11241\study1_positive\image1.png")
image = image.convert('RGB')
 # Convert PIL Image to numpy array
image_np = np.array(image)


        
image_gray = rgb2gray(image_np)
        
        
        
image_gray = exposure.equalize_adapthist(image_gray, clip_limit=0.02)

        
image_butterworth = butterworth(image_gray)

        

image_gaussian = gaussian(image_gray)
        
        # Combine the two images into one (stack them along the last axis)
        # Create a 3 channel image from the butterworth and gaussian filtered images
image_combined = np.stack([image_butterworth, image_gaussian, image_gray], axis=-1)
        
image_combined = (image_combined - np.min(image_combined)) / (np.max(image_combined) - np.min(image_combined))
        
        # Convert to 8-bit image for PIL
image_combined = img_as_ubyte(image_combined)
        
        # Convert numpy array back to PIL Image
image = Image.fromarray(image_combined)


transform = transforms.Compose([
    v2.Resize((224, 224)),
    v2.ToTensor(),
])
image = transform(image)
transformed_image = image.unsqueeze(0)

In [40]:
with torch.no_grad():
    googleNet_output = googleNet(transformed_image)
    googleNet_proba = F.softmax(googleNet_output, dim=1)

    # _, predicted_indices = torch.max(googleNet_output, 1)
    # predicted_index = predicted_indices.item()  # Convert tensor to Python number
    

In [41]:
print(googleNet_proba)

tensor([[0.1150, 0.8850]])


In [42]:
with torch.no_grad():
    swin_output = swin(transformed_image)
    logits = swin_output.logits 
    swin_proba = F.softmax(logits, dim=1)

    _, predicted_indices = torch.max(logits, 1)
    predicted_index = predicted_indices.item()  # Convert tensor to Python number

In [43]:
print(swin_proba)

tensor([[0.9682, 0.0318]])


In [44]:
swin_proba_np = swin_proba.numpy()
googleNet_proba_np = googleNet_proba.numpy()


In [45]:
probabilities = np.array([swin_proba_np, googleNet_proba_np])
print(probabilities)


[[[0.9682156  0.03178439]]

 [[0.11498407 0.88501596]]]


In [46]:
average_probabilities = np.mean(probabilities, axis=0)
ensemble_prediction = np.argmax(average_probabilities)  


In [47]:
average_probabilities

array([[0.5415998, 0.4584002]], dtype=float32)

In [48]:
ensemble_prediction

0

In [49]:
if ensemble_prediction == 0:
        print('negative')
elif ensemble_prediction == 1:
        print('postive')

negative
